In [1]:
import os
import pandas as pd
from collections import OrderedDict

from bokeh.io import output_file, show
from bokeh.models.map_plots import GMapPlot, GMapOptions
from bokeh.models.sources import ColumnDataSource
from bokeh.models.ranges import DataRange1d
from bokeh.models.tools import PanTool, WheelZoomTool, BoxSelectTool, HoverTool, CrosshairTool
from bokeh.models.markers import Circle

In [3]:
wells_aggregated = pd.read_csv('../data/wells_aggregated.csv')
wells_aggregated.head()

,Unnamed: 0,api_label,utm_x,utm_y,max_md,max_tvd,operator,well_title,latitude,longitude,topic_num,topic_name,5,spud_date
0,0,05-123-05002,510651,4428691,9441,0,CRESTONE PEAK RESOURCES OPERATING LLC,1 GRENEMEYER-WAGNER,40.008346,-104.875207,5,permit,2.0,1955-05-07
1,1,05-123-05214,514936,4479649,4650,4650,BROOKS EXPLORATION INC,1-19R WINDER,40.467382,-104.823816,5,permit,1.0,NaN
2,2,05-123-07520,504322,4440922,0,0,MARTIN OIL SERVICE INC,5 WALSH,40.118600,-104.949279,5,permit,1.0,NaN
3,3,05-123-07557,527820,4452419,7808,0,KERR MCGEE OIL & GAS ONSHORE LP,2 STATE,40.221732,-104.673025,5,permit,3.0,1972-10-07
4,4,05-123-07899,511492,4441722,4760,0,TOP OPERATING COMPANY,1-23 KNOX,40.125740,-104.865122,5,permit,1.0,1973-10-14


In [4]:
api_label = wells_aggregated['api_label'].tolist()
max_md = wells_aggregated['max_md'].tolist()
max_tvd = wells_aggregated['max_tvd'].tolist()
operator = wells_aggregated['operator'].tolist()
spud_date = wells_aggregated['spud_date'].tolist()
well_title = wells_aggregated['well_title'].tolist()
doc_type = wells_aggregated['topic_name'].tolist()
doc_count = wells_aggregated['5'].tolist()

In [5]:
map_options = GMapOptions(lat=40.30, lng=-104.20, map_type='roadmap', zoom=8)

plot = GMapPlot(x_range=DataRange1d(), y_range=DataRange1d(), map_options=map_options,
                plot_width=600, plot_height=600)

plot.title.text = 'Weld County Wells'

plot.api_key = os.environ['GMAP_ACCESS_KEY_ID']

# hover = plot.select(dict(type=HoverTool))

# hover.tooltips = OrderedDict([('API_Label','API_Label')])

source = ColumnDataSource(data=dict(lat=wells_aggregated['latitude'].tolist(),
                                    lon=wells_aggregated['longitude'].tolist(),
                                    api_label=api_label, max_md=max_md, max_tvd=max_tvd,
                                    operator=operator, spud_date=spud_date,
                                    well_title=well_title, doc_type=doc_type,
                                    doc_count=doc_count))

circle = Circle(x="lon", y="lat", size=10, fill_color='red', fill_alpha=0.8, line_color=None)
plot.add_glyph(source, circle)

plot.add_tools(PanTool(), WheelZoomTool(), BoxSelectTool(), HoverTool(),CrosshairTool())

output_file("../app/docmap/docmapapp/templates/docmapapp/includes/gmap_plot.html")

hover = plot.select(dict(type=HoverTool))
#hover.snap_to_data = False	# not supported in new bokeh versions
hover.tooltips = OrderedDict( [('API Label', '@api_label'), ('Latitude', '@lat'), ('Longitude', '@lon'), 
                               ('Operator', '@operator'), ('Spud Date', '@spud_date'), ('Well Name', '@well_title'), 
                               ('Max MD', '@max_md'), ('Max TVD', '@max_tvd'),
                               ('Document Type/Count', '@doc_type (@doc_count)')
                              ] )

show(plot)